In [1]:
import os, torch, numpy as np
# from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.svm import SVC 
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    accuracy_score,
    recall_score,
    f1_score,
    classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

RuntimeError: operator torchvision::nms does not exist

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from preprocess.data_process import get_dataloaders

train_loader, val_loader, test_loader = get_dataloaders()

images, labels = next(iter(train_loader))
print(images[0].shape)
print(labels[0])

torch.Size([3, 80, 80])
tensor(1)


In [ ]:
#Build frozen ResNet‑18 feature extractor
base_model = models.resnet18(pretrained=True)
feature_extractor = nn.Sequential(*list(base_model.children())[:-1]).to(device)
feature_extractor.eval()


/home/public/sgurpree/miniconda3/envs/G-CV-I_Final/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/public/sgurpree/miniconda3/envs/G-CV-I_Final/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/undergrad/2026/sgurpree/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 102MB/s] 


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
from torchinfo import summary
summary(base_model, input_size=(64, 3, 80, 80))

NameError: name 'images' is not defined

In [ ]:
@torch.no_grad()
def extract_features(loader):
    feats, labs = [], []
    for imgs, labels in loader:
        imgs = imgs.to(device)
        out  = feature_extractor(imgs)        # → [B,512,1,1]
        out  = out.view(out.size(0), -1)      # → [B,512]
        feats.append(out.cpu())
        labs.append(torch.tensor(labels))
    return torch.cat(feats, dim=0), torch.cat(labs, dim=0)

train_feats, train_labels = extract_features(train_loader)
test_feats,  test_labels  = extract_features(test_loader)
print(f"Train feats: {train_feats.shape}, Train labels: {train_labels.shape}")
print(f"Test  feats: {test_feats.shape},  Test labels: {test_labels.shape}")


In [ ]:
# Cell 9: Pure‑PyTorch KNN and evaluation
def knn_predict(test_feats, train_feats, train_labels, k=5):
    dists    = torch.cdist(test_feats, train_feats)  # [N_test,N_train]
    idxs     = dists.topk(k, largest=False).indices  # [N_test,k]
    knn_labs = train_labels[idxs]                    # [N_test,k]
    preds    = []
    for row in knn_labs:
        vals, counts = row.unique(return_counts=True)
        preds.append(vals[counts.argmax()].item())
    return torch.tensor(preds)

k     = 5
preds = knn_predict(test_feats, train_feats, train_labels, k)
acc   = (preds == test_labels).float().mean().item()
print(f"{k}-NN test accuracy: {acc * 100:.2f}%")


In [ ]:
# Convert features and labels from tensors to numpy
X_train = train_feats.numpy()
y_train = train_labels.numpy()
X_test  = test_feats.numpy()
y_test  = test_labels.numpy()

# Train an SVM
svm = SVC(kernel='linear') 
svm.fit(X_train, y_train)

# Predict
svm_preds = svm.predict(X_test)

In [ ]:
# KNN Report
print("KNN (k=5) Classification Report:")
print(classification_report(test_labels, preds))

# KNN Confusion Matrix
cm_knn = confusion_matrix(test_labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Greens", xticklabels=np.unique(test_labels), yticklabels=np.unique(test_labels))
plt.title("KNN Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# SVM Report
print("SVM (linear) Classification Report:")
print(classification_report(test_labels, svm_preds))

# SVM Confusion Matrix
cm_svm = confusion_matrix(test_labels, svm_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_svm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(test_labels), yticklabels=np.unique(test_labels))
plt.title("SVM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()